In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from mpi4py import MPI
# Wider cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Make sure folder for images exist
import os
try:
    os.mkdir('imgs/')
except FileExistsError as err:
    pass

# Jupyter magic
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Function for which we compute integral
def f(x):
    return 1 / x

# Analytical integral
def g(x):
    return np.log(x)

def integrate(f, g=None, limits=(1, 2), nodes=100):
    """Compute function integral by using trapezoid rule
    Args:
        f: function to integrate
        g: analytical integral, for self check and error estimation
        limits: (a, b)
        nodes: # of points to split the limit
    """
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    all_nodes = np.linspace(limits[0], limits[1], num=nodes)
    left, right = rank * nodes // size, (rank + 1) * nodes // size
    individual_nodes = list(all_nodes[left:right]) # convert to Python object
    if rank == size - 1:
        individual_nodes = all_nodes[left:] # leftmost process take the rest
    print(f'My rank: {rank}, world size: {size}, {individual_nodes}')
    comm.Barrier()
    partial_result = [(x, f(x)) for x in individual_nodes]
    
    data = comm.gather(data, root=0)
    
    sendbuf = np.empty(2, dtype = 'i')
    sendbuf[0] = 0
    sendbuf[1] = rank
    
    # Recieve data
    if rank == 0:
        recvbuf = np.empty(2, dtype = 'i')
    else:    
        recvbuf = None
        
    if rank == 0:
        printf(f"{nodes} trapezoids, [a, b] = [{limits}], result = {total_integral}")


#    /* Shut down MPI */
#    MPI_Finalize();


In [2]:
!mpirun -n 10 python scatter.py

/bin/sh: mpirun: command not found
